In [1]:
import os
import random
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [3]:
# Directory where videos are stored
DATA_DIR = "data/sport"

# Mapping classes
classes = sorted(os.listdir(DATA_DIR))  # Assuming folder names are class names
print("Classes:", classes)

# Load videos and labels
data = []
labels = []

for category in classes:
    videos = os.listdir(os.path.join(DATA_DIR, category))
    for video in videos:
        video_path = os.path.join(DATA_DIR, category, video)
        # You might want to extract features or frames from videos here
        # For example, capturing one frame for simplicity
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (224, 224))  # Resizing to a fixed size
            data.append(frame)
            labels.append(classes.index(category))
        cap.release()

data = np.array(data)
labels = np.array(labels)


Classes: ['barbell biceps curl', 'bench press', 'chest fly machine', 'deadlift', 'decline bench press', 'hammer curl', 'hip thrust', 'incline bench press', 'lat pulldown', 'lateral raise', 'leg extension', 'leg raises', 'plank', 'pull Up', 'push-up', 'romanian deadlift', 'russian twist', 'shoulder press', 'squat', 't bar row', 'tricep Pushdown', 'tricep dips']


In [4]:
# Normalize pixel values
data = data / 255.0

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=len(classes))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [6]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [7]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
17/17 [==============================] - 11s 560ms/step - loss: 4.0316 - accuracy: 0.1036 - val_loss: 2.9058 - val_accuracy: 0.0763
Epoch 2/10
17/17 [==============================] - 8s 500ms/step - loss: 2.4971 - accuracy: 0.2994 - val_loss: 2.5106 - val_accuracy: 0.3053
Epoch 3/10
17/17 [==============================] - 9s 511ms/step - loss: 1.2555 - accuracy: 0.6891 - val_loss: 2.2965 - val_accuracy: 0.4046
Epoch 4/10
17/17 [==============================] - 10s 586ms/step - loss: 0.3161 - accuracy: 0.9539 - val_loss: 2.2295 - val_accuracy: 0.4656
Epoch 5/10
17/17 [==============================] - 9s 547ms/step - loss: 0.0654 - accuracy: 0.9904 - val_loss: 2.6083 - val_accuracy: 0.4427
Epoch 6/10
17/17 [==============================] - 10s 577ms/step - loss: 0.0322 - accuracy: 0.9942 - val_loss: 3.0086 - val_accuracy: 0.4046
Epoch 7/10
17/17 [==============================] - 9s 537ms/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 3.0532 - val_accuracy: 0.4275
Epo

In [8]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy*100}%")


5/5 [==============================] - 1s 115ms/step - loss: 3.2083 - accuracy: 0.4656
Test accuracy: 46.56488597393036%


In [13]:
model_save_path = "sport_model.h5"
model.save(model_save_path)